In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [60]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]
#dodac exponential

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
 -0.00474491  -0.0385991     0.107446    …  -0.00456969  0.130865
  0.0444241   -0.000217229  -0.00158954      0.0114042   0.00881925
  0.0113698    0.000614226   0.00277025      0.00970774  0.00534032

[:, :, 3] =
  0.0103612    0.0562051   -0.0587965   …   0.101234      0.0793763
 -0.00301889  -0.0256467   -0.072688       -4.23628e-5   -0.0356423
  0.00842737   0.00319842  -0.00404637      0.000516848   0.00394277

;;; … 

[:, :, 59] =
 -0.0440495   0.0440722  -0.0528162  -0.0901498  …  -0.00235045   0.0528629
 -0.0272888   0.0418141   0.0719469  -0.0532693      0.0541974    0.00559155
 -0.00794691  0.0101629  -0.0131857  -0.0151771      0.00494441  -0.00639199

[:, :, 60] =
 -0.00832728   -0.0662535   -0.0684446   …  0.051945

In [4]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [61]:
returns_scenarios

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
 -0.00474491  -0.0385991     0.107446    …  -0.00456969  0.130865
  0.0444241   -0.000217229  -0.00158954      0.0114042   0.00881925
  0.0113698    0.000614226   0.00277025      0.00970774  0.00534032

[:, :, 3] =
  0.0103612    0.0562051   -0.0587965   …   0.101234      0.0793763
 -0.00301889  -0.0256467   -0.072688       -4.23628e-5   -0.0356423
  0.00842737   0.00319842  -0.00404637      0.000516848   0.00394277

;;; … 

[:, :, 59] =
 -0.0440495   0.0440722  -0.0528162  -0.0901498  …  -0.00235045   0.0528629
 -0.0272888   0.0418141   0.0719469  -0.0532693      0.0541974    0.00559155
 -0.00794691  0.0101629  -0.0131857  -0.0151771      0.00494441  -0.00639199

[:, :, 60] =
 -0.00832728   -0.0662535   -0.0684446   …  0.051945

In [62]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)

3×100000 Matrix{Float64}:
  0.113711    -0.0238247  -0.0874192  …   0.224209   0.0245357  0.49671
  0.00688754  -0.012715   -0.0741254     -0.253296   0.0735092  0.11449
 -0.00416346   0.0388384   0.03989       -0.0262923  0.023289   0.0525735

In [63]:
returns_scenarios_ann = zeros(3,100_000,15)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = annualise(returns_scenarios[asset,:,2:end],4)
end
returns_scenarios_ann

3×100000×15 Array{Float64, 3}:
[:, :, 1] =
  0.0872978   0.190534    0.00402934  …  0.128686   0.173841    0.281111
 -0.0406462  -0.105149    0.0061346      0.0868686  0.0531541  -0.130524
  0.0136027   0.00102579  0.00707769     0.0028494  0.0288886  -0.000939727

[:, :, 2] =
 0.0507943   0.0589714    0.0333336   …  0.286769   -0.0867655  0.10659
 0.0411089   0.0489743    0.0587381      0.0862566  -0.0184568  0.10236
 0.0056767  -0.000782757  0.00857194     0.0127161  -0.0208908  0.0355812

[:, :, 3] =
 -0.363453   0.137823    0.149672    …  -0.206404    0.2293      -0.0151439
  0.117821   0.0690974   0.00695729     -0.175436   -0.0303123    0.0444029
 -0.0180873  0.0193933  -0.0195828      -0.0172528  -0.00321468   0.0467988

;;; … 

[:, :, 13] =
  0.0211434    0.087946     0.24788    0.0858162  …  -0.00289898  -0.00924662
  0.0409134   -0.0267572    0.0759726  0.117595       0.0566176    0.00308006
 -0.00588223   0.00979692  -0.0437098  0.0181884      0.00118032  -0.0408929

[:, :, 

In [78]:
quantile(returns_scenarios_ann[1,:,5],[0.1,0.5,0.9])

3-element Vector{Float64}:
 -0.1407979202612639
  0.05188617213713301
  0.24378691415156697

In [69]:
lattice = cluster(s,7)

lattice[1]

7×3 transpose(::Matrix{Float64}) with eltype Float64:
  0.0671815  -0.102905   -0.0273199
 -0.0805466   0.14173     0.0291884
  0.194932    0.0808928   0.0366266
  0.283995   -0.0663927   0.00237186
  0.108179    0.0635744  -0.00553659
 -0.153909    0.0241577  -0.00973469
  0.0199797  -0.031112    0.0178776

In [7]:
s

3×100000 Matrix{Float64}:
 -0.0264132   -0.0809205  0.163329   …  -0.15529    -0.151671     0.157315
 -0.0252872   -0.120145   0.0975189      0.0748333  -0.0540429   -0.0179953
  0.00900834   0.0385699  0.0137708     -0.0010503  -0.00923074   0.0371984

In [8]:
clusters_probs(lattice[2])

9-element Vector{Float64}:
 0.12283
 0.09447
 0.08931
 0.15239
 0.11576
 0.0919
 0.12432
 0.11663
 0.09239

In [ ]:
states = stage_states([])
@test states .== []

In [72]:
res = generate_lattice(returns_scenarios_ann, 7,5)


Lattice([0.10854567050570815 0.31912626239509195 … -0.007867351525814276 0.15512953177090308; -0.06050387245653244 0.30003258110439646 … 0.15191842250267196 0.2762404148827719; … ; 0.1946028267686263 0.1503841620235178 … -0.022531681955411553 0.11178690248685112; 0.08089020703099 0.15065087680409414 … 0.28012074878537674 0.058948733807880835;;; -0.06434404354214415 -0.06498177285918763 … 0.06733694244075168 0.028918877780019295; 0.00020380106446464888 -0.09394466590559293 … 0.02648641030922739 0.020790734708253884; … ; -0.1197781415631702 0.028537396472369102 … -0.05252528779359349 -0.06322965798460962; -0.10411697498026797 0.026520378274771036 … 0.018689319740240837 0.07810691873770446;;; 0.020128357141610237 0.009772515031111782 … 0.025748060777008967 0.004909992125914771; 0.010693344269351118 0.0028856227885003857 … 0.004356102644510805 0.02459991843621396; … ; -0.005364446414769606 0.004687359320677639 … 0.0015034788794136764 0.0204748366469085; -0.005695877617298568 0.004324005412

In [73]:
res.states[5,:,:]

7×3 Matrix{Float64}:
  0.0808902  -0.104117    -0.00569588
  0.150651    0.0265204    0.00432401
  0.297221   -0.0956251    0.00243344
 -0.0615204   0.00254379   0.0113371
  0.114519   -0.0658182    0.020042
  0.280121    0.0186893    0.024081
  0.0589487   0.0781069    0.0300945

In [46]:
res.probs[2]

BoundsError: BoundsError: attempt to access 1-element Vector{Matrix{Float64}} at index [2]